# Data importing

In [1]:
import numpy as np

x = [[1, 0.6, 0.1],
     [0, -0.4, 0.8],
     [0, 0.2, 0.5],
     [1, 0.4, -0.1]]

In [2]:
coef1 = 0.5
coef2 = 0.5

p1 = 0.3
p2 = 0.7

mean1 = [1, 1]
cov_matrix1 = [[2, 0.5], [0.5, 2]]

mean2 = [0, 0]
cov_matrix2 = [[1.5, 1], [1, 1.5]]

# E-Step

P(x|c)

In [28]:
from scipy.stats import multivariate_normal

p_x_c1 = []
for i in range(len(x)):
    if x[i][0] == 1:
        p = p1
    else:
        p = 1-p1
    p_x_c1.append(p * multivariate_normal.pdf(x[i][1:], mean1, cov_matrix1))

p_x_c1

[0.2314743376774436,
 1.266332483325112,
 1.4381104036459127,
 0.02076522613490563]

In [29]:
p_c1_x_prop = np.multiply(coef1, p_x_c1)
p_c1_x_prop

array([0.08938788, 0.48901652, 0.55535158, 0.00801886])

In [30]:
p_x_c2 = []
for i in range(len(x)):
    if x[i][0] == 1:
        p = p2
    else:
        p = 1-p2
    p_x_c2.append(p * multivariate_normal.pdf(x[i][1:], mean2, cov_matrix2))

p_x_c2

[0.9495425230090517,
 0.08873672123767427,
 0.4541744972869781,
 0.7233119821229741]

In [31]:
p_c2_x_prop = np.multiply(coef2, p_x_c2)
p_c2_x_prop

array([0.58286001, 0.05446948, 0.27878704, 0.44399236])

P(c|x)

In [32]:
p_c1_x = np.divide(p_c1_x_prop, np.add(p_c1_x_prop, p_c2_x_prop))
p_c1_x

array([0.13296863, 0.89977759, 0.66577852, 0.01774039])

In [33]:
p_c2_x = np.divide(p_c2_x_prop, np.add(p_c1_x_prop, p_c2_x_prop))
p_c2_x

array([0.86703137, 0.10022241, 0.33422148, 0.98225961])

N1 and N2

In [34]:
N1 = sum(p_c1_x)
N1

1.716265128604998

In [35]:
N2 = sum(p_c2_x)
N2

2.2837348713950023

π1 and π2

In [36]:
new_coef1 = N1/(N1+N2)
new_coef1

0.4290662821512495

In [37]:
new_coef2 = N2/(N1+N2)
new_coef2

0.5709337178487506

p1 and p2

In [38]:
new_p1 = 0
for i in range(len(x)):
    new_p1 += p_c1_x[i] * x[i][0]

new_p1 = new_p1 / N1
new_p1

0.08781220208246035

In [39]:
new_p2 = 0
for i in range(len(x)):
    new_p2 += p_c2_x[i] * x[i][0]

new_p2 = new_p2 / N2
new_p2

0.8097660559740163

mean1 and mean2

In [40]:
new_mean1 = np.array([[0], [0]])
for i in range(len(x)):
    new_mean1 = np.add(new_mean1, p_c1_x[i] * np.vstack(x[i][1:]))

new_mean1 = np.divide(new_mean1, N1)
new_mean1

array([[-0.08150139],
       [ 0.62008727]])

In [41]:
new_mean2 = np.array([[0], [0]])
for i in range(len(x)):
    new_mean2 = np.add(new_mean2, p_c2_x[i] * np.vstack(x[i][1:]))

new_mean2 = np.divide(new_mean2, N2)
new_mean2

array([[0.41155303],
       [0.10323696]])

cov_matrix1 and cov_matrix2

In [42]:
new_covmatrix1 = np.array([[0, 0], [0, 0]])
for i in range(len(x)):
    new_covmatrix1 = np.add(new_covmatrix1, 
        np.dot(
            p_c1_x[i] * np.subtract(np.vstack(x[i][1:]), new_mean1),
            np.transpose(np.subtract(np.vstack(x[i][1:]), new_mean1))
        )
    )

new_covmatrix1 = np.divide(new_covmatrix1, N1)
new_covmatrix1

array([[ 0.12230189, -0.07419944],
       [-0.07419944,  0.04888017]])

In [43]:
new_covmatrix2 = np.array([[0, 0], [0, 0]])
for i in range(len(x)):
    new_covmatrix2 = np.add(new_covmatrix2, 
        np.dot(
            p_c2_x[i] * np.subtract(np.vstack(x[i][1:]), new_mean2),
            np.transpose(np.subtract(np.vstack(x[i][1:]), new_mean2))
        )
    )

new_covmatrix2 = np.divide(new_covmatrix2, N2)
new_covmatrix2

array([[ 0.04899327, -0.03632106],
       [-0.03632106,  0.06211355]])

In [44]:
np.linalg.inv(new_covmatrix2)

array([[36.0302774 , 21.06879999],
       [21.06879999, 28.41958383]])

# Ex2 - X_new = [1, 0.3, 0.7]

In [45]:
x_new = [1, 0.3, 0.7]

In [46]:
if x_new[0] == 1:
    p = new_p1
else:
    p = 1-new_p1
p_xnew_c1 = p * multivariate_normal.pdf(x_new[1:], mean1, cov_matrix1)

p_xnew_c1

0.002377565746898642

In [47]:
p_c1_xnew_prop = new_coef1 * p_xnew_c1
p_c1_xnew_prop

0.001020133295591959

In [48]:
if x_new[0] == 1:
    p = new_p2
else:
    p = 1-new_p2
p_xnew_c2 = p * multivariate_normal.pdf(x_new[1:], mean2, cov_matrix2)

p_xnew_c2

0.05541300469173034

In [49]:
p_c2_xnew_prop = new_coef2 * p_xnew_c2
p_c2_xnew_prop

0.03163715278581986

In [50]:
p_c1_xnew = p_c1_xnew_prop / np.add(p_c1_xnew_prop, p_c2_xnew_prop)
p_c1_xnew

0.03123754046949443

In [51]:
p_c2_xnew = p_c2_xnew_prop / np.add(p_c1_xnew_prop, p_c2_xnew_prop)
p_c2_xnew

0.9687624595305055

NOTE: If you want to change to the newly computed values and re-run the code above, uncomment and run the block below.

To revert the changes, restart the python kernel. 

In [52]:
coef1 = new_coef1
coef2 = new_coef2

p1 = new_p1
p2 = new_p2

mean1 = np.concatenate(np.transpose(new_mean1))
mean2 = np.concatenate(np.transpose(new_mean2))

cov_matrix1 = new_covmatrix1
cov_matrix2 = new_covmatrix2

mean1

array([-0.08150139,  0.62008727])